In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# CRUD FILE
from AnimalShelter import AnimalShelter

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CRUD operations for Animal collection in MongoDB
from pymongo import MongoClient
from bson.objectid import ObjectId

# MongoDB Connection Variables
USER = 'aacuser'
PASS = 'password'
HOST = 'nv-desktop-services.apporto.com'
PORT = 31918
DB = 'aac'
COL = 'animals'

# Connect to MongoDB
shelter = AnimalShelter(USER, PASS)

# DataFrame
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)

# Query Mappings
QUERY_MAPPINGS = {
    "Water": {
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": {"$in": ["Intact Female"]},
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
},
    "Mountain or Wilderness": {
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": {"$in": ["Intact Male"]},
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Disaster or Individual Tracking": {
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": {"$in": ["Intact Male"]},
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    },
    "Reset": {}  # Empty query returns all documents
}

# Setting up the Dashboard Layout / View
app = JupyterDash(__name__)
image_filename = 'grazioso-logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Ryans Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div(
        # Interactive filtering options
        dcc.Dropdown(
            id='rescue-type-dropdown',
            options=[
                {"label": "Water Rescue", "value": "Water"},
                {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness"},
                {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"},
                {"label": "Reset", "value": "Reset"}
            ],
            value='Reset',  # Default value
            clearable=False  # Prevents user from removing the selection
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',     # single row selection
        selected_rows=[1]            # By default row 0 is selected
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])

# Interaction Between Components / Controller
@app.callback(Output('datatable-id', 'data'),
              [Input('rescue-type-dropdown', 'value')])
def update_dashboard(selected_rescue_type):
    if selected_rescue_type == "Reset":
        results = list(shelter.read({}))  # Get all data for now
    else:
        query = QUERY_MAPPINGS[selected_rescue_type]  # Assign the query first
        results = list(shelter.read(query))

    if not results:
        return []

    # Convert ObjectId to string
    for record in results:
        record['_id'] = str(record['_id'])
    
    try:
        df_data = pd.DataFrame.from_records(results)
        data = df_data.to_dict('records')
        return data
    except Exception as e:
        print(f"Error during conversion: {e}")
        return []

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' in dff.columns:
        fig = px.pie(dff, names='breed', title='Preferred Animals')
        fig.update_layout(autosize=True)
        return [
            dcc.Graph(figure=fig)
        ]
    return []

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns:
        return [{
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns]
    return []

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, derived_virtual_selected_rows):
    if not viewData or not derived_virtual_selected_rows:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    row = derived_virtual_selected_rows[0]

    # Fetch latitude and longitude by column name
    lat_val = dff.at[row, 'location_lat']
    lon_val = dff.at[row, 'location_long']

    try:
        # Convert latitude and longitude to float
        lat = float(lat_val)
        lon = float(lon_val)
    except ValueError as e:
        print(f"Error converting lat or lon: {e}")
        return []
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1(dff.at[row, 'name']),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:21177/
